# Run Summa model on HPC

Structure for Unifying Multiple Modeling Alternative [(SUMMA)](https://ral.ucar.edu/projects/summa) is a hydrologic modeling approach that is built on a common set of conservation equations and a common numerical solver, which together constitute the structural core of the model. Different modeling approaches can then be implemented within the structural core, enabling a controlled and systematic analysis of alternative modeling options, and providing insight for future model development.

This notebook demonstrates how to harmonize different libraries and tools provided in the CyberGIS-Jupyter environment to run SUMMA model with a High Performance Computer (HPC) resource (Lyu 2019). The SUMMA Model instance used is a part of the Clark et al., (2015) study, and explored the impact of different stomatal resistance parameterizations on total evapotranspiration (ET) in the Reynolds Mountain East catchment in southwestern Idaho.

Lyu, F., Yin, D., Padmanabhan, A., Choi, Y., Goodall, J. L., Castronova, A., Tarboton, D., Wang, S. (2019) “Bridging Reproducible Hydrological Modeling with CyberGIS-Jupyter: A Case Study on SUMMA”. In: Proceedings of Practice and Experience in Advanced Research Computing (PEARC19), July 28-August 1, 2019, Chicago, IL, USA. https://dl.acm.org/citation.cfm?id=3333052

Clark, M. P., B. Nijssen, J. Lundquist, D. Kavetski, D. Rupp, R. Woods, E. Gutmann, A. Wood, D. Gochis, R. Rasmussen, D. Tarboton, V. Mahat, G. Flerchinger, and D. Marks, 2015: A unified approach for process-based hydrologic modeling: Part 2. Model implementation and example applications. Water Resources Research, 51, doi: 10.1002/2015WR017200



# Notebook Outline
- [Retrieve Resources](#resources)
- [Unzip model instance package](#unzip)
- [Install SUMMA model](#install)
- [Use PySumma to configure model parameters](#build)
- [Submit model](#submit)
- [Check model output](#check)
- [Plot](#plot)
- [Cleanup](#cleanup)

<a id='resources'></a>
## Retrieve a Summa model instance resource from HydroShare

This is the resource id of a HydroShare composite resource that contains a SUMMA model instance/. For more info on this resource please refer to https://www.hydroshare.org/resource/1f3f310af8364d2aa3e6a9459152a21c/)

In [ ]:
resource_id = '1f3f310af8364d2aa3e6a9459152a21c'

We use HydroShare REST API Python client (hs_restclient) to retrieve the above resource onto the notebook server

In [ ]:
import json
import os
from hs_restclient import HydroShare, HydroShareAuthBasic
auth = HydroShareAuthBasic("cybergis", "demo")
hs = HydroShare(auth=auth)
base_dir = os.path.abspath('/home/jovyan/work')
download_dir = os.path.join(base_dir, 'Downloads')
!mkdir -p {download_dir}
hs.getResource(resource_id, destination=download_dir, unzip=True)

<a id='unzip'></a>
## Unzip model instance package

Create a temporary local folder and unzip the summa instance

In [ ]:
import os
#Unzip model file
model_folder_name = "SummaModel_ReynoldsAspenStand_StomatalResistance_sopron"
content_folder = os.path.join(download_dir ,"{}/{}/data/contents".format(resource_id, resource_id))
file_manger_rel_path = "settings/summa_fileManager_riparianAspenSimpleResistance.txt"
import tempfile
workspace_dir = os.path.join(base_dir, 'workspace')
!mkdir -p {workspace_dir}
unzip_dir = tempfile.mkdtemp(dir=workspace_dir)
!cd {content_folder} && unzip -o {model_folder_name}.zip -d {unzip_dir}
print("Unzipping Done")

<a id='install'></a>
## "Install" SUMMA model on Jupyter server (fix broken path in summa file_manager)

By default, the file_manager file of the Summa instance does not contain local path. Need to use script to install it locally.

In [ ]:
model_source_folder_path = os.path.join(unzip_dir, model_folder_name)
!cd {model_source_folder_path} && chmod +x ./installTestCases_local.sh
!cd {model_source_folder_path} && ./installTestCases_local.sh

<a id='build'></a>
## Use PySumma to configure model parameters

We set Start and Finish points for the simulation and set Stomatal Resistance type to be 'BallBerry'

In [ ]:
from pysumma import Simulation

# path to the SUMMA filemanager file on Jupyter
file_manager = os.path.join(model_source_folder_path, file_manger_rel_path)

# Create pySUMMA Simulation Object
executable = ""
S = Simulation(executable, file_manager)

# Configure the model
S.decisions['simulStart'].value = "2006-07-01 00:00"
S.decisions['simulFinsh'].value = "2007-08-20 00:00"
S.decisions['stomResist'].value = 'BallBerry'
# Save configiuration to disk
S._write_configuration()
print(S)

<a id='submit'></a>
## Submit model to XSEDE Comet HPC using CyberGIS-Jupyter tool

We use cybergis lib to submit the model to HPC as a job

In [ ]:
from cybergis import summaUI

s = summaUI(model_source_folder_path, file_manger_rel_path, 
            workspace_dir)

Walltime (hour) is the maximum duration of time the job is allowed to run on the HPC before it gets killed. Nodes indicate the number of CPU cores will be allocated for this job. If multiple model runs are configured (see ensemble example), CyberGIS lib will run the job over the assigned CPUs in parallel. In this case where only a single model run is configured, we just need to assign it with 1 node and use the default Walltime (1 hour).

In [ ]:
s.runSumma()

Once job finishes execution on HPC, cybergis lib will retrieve the results back to notebook server. In this case there will be only one netcdf file in the output.

In [ ]:
job_dir = os.path.join(workspace_dir, 
                            "{}/output".format(s.job_local_id))
summa_output = os.path.join(job_dir, 
                            "vegImpactsTranspire_output_test_timestep.nc")
print(summa_output)

<a id='check'></a>
## Check model output  -- NetCDF file

Use xarray to check the metadata of the nc file

In [ ]:
import xarray as xr
output_info = xr.open_dataset(summa_output)
output_info

<a id='plot'></a>
## Plot time series for total evapotranspiration

Plot Total Evapotranspiration over time

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
x = output_info['time']
y = (output_info['scalarCanopyTranspiration'] + \
     output_info['scalarCanopyEvaporation'] + \
     output_info['scalarGroundEvaporation'])*3600

plt.figure(figsize=(15, 5))
plt.plot(x, y, color='grey', linestyle='solid', markersize=0)

    # Get the current axis of the plot and
    # set the x and y-axis labels
ax = plt.gca()
ax.set_ylabel("Total ET (kg m-2 h-1)")
ax.set_xlabel('Date')
ax.grid(True)

ax.set_title('SUMMA Plot for Total ET')
output_info.close()

<a id='cleanup'></a>
## Cleanup

This step is optional, which will remove all model outputs on notebook server

In [ ]:
! rm -rvf {unzip_dir} {job_dir}

## Done